In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from datetime import datetime

In [2]:
def getULCustNum(currentTime):
    def getTimeResult(currentTime):
        parts = currentTime.split(' ')
    #     print(parts)
        time_now = parts[0] + ' ' + parts[1]
        result = datetime.strptime(time_now, '%Y/%m/%d %H:%M:%S')
        return result
    def getMinutesFromZero(dt_obj):
        hour = dt_obj.hour
        mins = dt_obj.minute
        return hour * 60 + mins
    def getULCustNumIn(currentTime):
        result = getTimeResult(currentTime)
        # dir(result)
        week_day = result.weekday()
        mins_for_input_time = getMinutesFromZero(result)
        # print(mins_for_input_time)
        # result_df = df.query('(day_time < mins_for_input_time) and (day_time - 15 < mins_for_input_time))')

#         print(df[df['day_time'] <= mins_for_input_time].shape)
        upper = df[df['day_time'] <= mins_for_input_time].iloc[-1, (week_day+1)]
        lower = df[df['day_time'] >= mins_for_input_time].iloc[0, (week_day+1)]
                
        return upper, lower
    df = pd.read_excel('./utils/港口人数.xlsx')
    # get a new columns in df
    df['day_time'] = pd.to_datetime(df['时刻'], format='%H:%M')
    df['day_time'] = df['day_time'].apply(getMinutesFromZero)
    # df
    return getULCustNumIn(currentTime)

In [3]:
filenames = os.listdir('./track_exp/')

In [4]:
### calculate time function
def convert_timedelta(duration):
    days, seconds = duration.days, duration.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = (seconds % 60)
    return hours, minutes, seconds
def getTimeResult(currentTime):
    parts = currentTime.split(' ')
#     print(parts)
    time_now = parts[0] + ' ' + parts[1]
    result = datetime.strptime(time_now, '%Y/%m/%d %H:%M:%S')
    return result
def getMinutes(begin_time, end_time):
    result_begin = getTimeResult(begin_time)
    result_end = getTimeResult(end_time)
    delta = result_end - result_begin
    hours, minutes, seconds = convert_timedelta(delta)
    mins = hours * 60 + minutes
    return mins

In [5]:
def InAirport(series):
    jd = series['jd']
    wd = series['wd']
    point = Point(jd, wd)
    vertex_lst_baoan = [
        [113.793451,22.656175],
        [113.801069,22.659776],
        [113.803225,22.653506],
        [113.815585,22.660177],
        [113.818783,22.655374],
        [113.826077,22.659043],
        [113.847062,22.627022],
        [113.817454,22.614612]
    ]
    
    baoan = Polygon(vertex_lst_baoan)
    result_baoan = baoan.contains(point)
    
    return result_baoan

In [6]:
class AirportPeriod:
#     def __init__(self, start_waitingTime, end_waitingTime, drop, pick, duration, cust_num):
    def __init__(self, start_waitingTime, end_waitingTime, drop, pick, duration):
        self.start_waitingTime = start_waitingTime
        self.end_waitingTime = end_waitingTime
        self.drop = drop ### whether this driver dropped someone to the airport
        self.pick = pick ### whether this driver picked someone in the airport
        self.duration = duration ### how long this driver waited
#         self.cust_num = cust_num ### volume of passengers

In [10]:
for file_name in filenames:
    car_name = file_name[:-3]
    try:
#     if True:
        df = pd.read_csv('./track_exp/%s'%file_name)
        df['InAirport'] = df.apply(InAirport, axis=1)

        inList = df[df['InAirport']].index.tolist()

        ### if never in the airport:
        if len(inList) == 0:
            continue

        ### insert below code here
        in_idx = inList[0]-1
        last_idx = inList[0]-1

#         period_df = pd.DataFrame({'StartWaiting':[], 'EndWaiting':[], 'drop':[], 'pick':[], 'duration':[],'CustomerNum':[]})
        period_df = pd.DataFrame({'StartWaiting':[], 'EndWaiting':[], 'drop':[], 'pick':[], 'duration':[]})
        ### get seperate in-airport intervals and write its info into a dataframe
        for idx in inList:
            if idx-last_idx != 1: ### left the airport in the last period, enter the airport in the next period

                InAirport_df = df[(df.index >= in_idx) & (df.index <= last_idx)]

                ### rule out taxi that is always loaded in the airport (only pass by)
                if InAirport_df[InAirport_df['status']==0].shape[0] == 0:
                    continue

                ### if init status is 1, it's dropping some shit, so init state should be when status is 0; if not, fuck this shit
                if InAirport_df.iloc[0]['status'] == 1:
                    drop = True
                    start_waitingTime = InAirport_df[InAirport_df['status']==0].iloc[0]['time']
                else:
                    drop = False
                    start_waitingTime = InAirport_df.iloc[0]['time']

                ### if final status is 1, it's picking up some shit; if not, the driver doesn't wait for shit
                if InAirport_df.iloc[-1]['status'] == 1:
                    pick = True
                    end_waitingTime = InAirport_df[InAirport_df['status']==0].iloc[-1]['time']
                else:
                    pick = False
                    end_waitingTime = InAirport_df.iloc[-1]['time']

                duration = getMinutes(start_waitingTime, end_waitingTime)
#                 cust_num = getULCustNum(start_waitingTime)[0] ### TODO: is this accurate?
                
#                 period = AirportPeriod(start_waitingTime, end_waitingTime, drop, pick, duration, cust_num)
                period = AirportPeriod(start_waitingTime, end_waitingTime, drop, pick, duration,)

                if duration>1:
                    period_df.loc[period_df.shape[0]+1] = {
                        'StartWaiting':period.start_waitingTime, 
                        'EndWaiting':period.end_waitingTime, 
                        'drop':period.drop, 
                        'pick':period.pick, 
                        'duration':period.duration,
#                         'CustomerNum':period.cust_num
                    }

                in_idx = idx ### set new period

            last_idx = idx
        if period_df.shape[0]!= 0:
            period_df.to_csv('./PeriodData/period_%s'%file_name)
        
    except:
        print(file_name,'I am a stupid fuck')
#     except:
#         if UnicodeDecodeError:
#             print(car_name)
#             continue
#         else:
#             print("Unknown shit fucked in file%"%file_name)
#             continue

gps_readme.txt I am a stupid fuck
.DS_Store I am a stupid fuck
data_cleaning.ipynb I am a stupid fuck
data_tocsv.ipynb I am a stupid fuck
